In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchsummary import summary  # Denne er fin for å printe ut informasjon om PyTorch-modeller

In [ ]:
# Loading the data as PyTorch Tensor Dataset
data_train = torch.load("data\localization_train.pt")
data_val = torch.load("data\localization_val.pt")
data_test = torch.load("data\localization_test.pt")
print("Type: ", type(data_train))

In [ ]:
# Convert data to PyTorch Tensors
train_imgs = torch.stack([img for img, label in data_train])
val_imgs = torch.stack([img for img, label in data_val])
test_imgs = torch.stack([img for img, label in data_test])
print(type(train_imgs[0]))

print("Size: ", train_imgs.size())